# Benchmark: pip vs uv on Mac M1

This notebook benchmarks installation speed and memory/CPU usage for `pip` and `uv` using a set of common Python packages. Results are shown as tables and charts.

In [ ]:
%pip install -q psutil matplotlib pandas

In [ ]:
import time
import subprocess
import psutil
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display

COMMON_PACKAGES = [
    'numpy', 'requests', 'pandas', 'scipy', 'flask'
]

def get_total_mem_and_cpu(proc):
    mem = proc.memory_info().rss if proc.is_running() else 0
    cpu = proc.cpu_percent(interval=0.1) if proc.is_running() else 0
    for child in proc.children(recursive=True):
        try:
            if child.is_running():
                mem += child.memory_info().rss
                cpu += child.cpu_percent(interval=0.1)
        except Exception:
            pass
    return mem / (1024*1024), cpu

def run_with_resource_monitor(cmd, timeout=300):
    process = subprocess.Popen(cmd, shell=True)
    proc = psutil.Process(process.pid)
    max_mem = 0
    max_cpu = 0
    start = time.time()
    try:
        while process.poll() is None:
            mem, cpu = get_total_mem_and_cpu(proc)
            if mem > max_mem:
                max_mem = mem
            if cpu > max_cpu:
                max_cpu = cpu
            time.sleep(0.1)
            if time.time() - start > timeout:
                process.kill()
                raise TimeoutError(f"Command timed out after {timeout}s")
        elapsed = time.time() - start
    except Exception as e:
        process.kill()
        raise e
    return elapsed, max_mem, max_cpu

def benchmark_installer(installer, packages):
    results = []
    env_dir = Path(f"/tmp/bench_{installer}")
    if env_dir.exists():
        subprocess.run(f"rm -rf {env_dir}", shell=True)
    subprocess.run(f"python3 -m venv {env_dir}", shell=True, check=True)
    pybin = env_dir / 'bin' / 'python'
    installer_bin = env_dir / 'bin' / installer
    subprocess.run(f"{pybin} -m pip install --upgrade pip", shell=True, check=True)
    if installer == 'uv':
        subprocess.run(f"{pybin} -m pip install uv", shell=True, check=True)
    for pkg in packages:
        subprocess.run(f"{pybin} -m pip uninstall -y {pkg}", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        if installer == 'uv':
            cmd = f"{installer_bin} pip install {pkg}"
        else:
            cmd = f"{installer_bin} install {pkg}"
        try:
            elapsed, max_mem, max_cpu = run_with_resource_monitor(cmd)
            results.append({'package': pkg, 'time_sec': elapsed, 'max_mem_mb': max_mem, 'max_cpu_pct': max_cpu, 'error': None})
        except Exception as e:
            results.append({'package': pkg, 'time_sec': None, 'max_mem_mb': None, 'max_cpu_pct': None, 'error': str(e)})
    return pd.DataFrame(results)

In [ ]:
pip_results = benchmark_installer('pip', COMMON_PACKAGES)
uv_results = benchmark_installer('uv', COMMON_PACKAGES)

## Results Table

In [ ]:
display(pip_results)
display(uv_results)

## Visual Comparison

In [ ]:
import numpy as np
labels = COMMON_PACKAGES
x = np.arange(len(labels))

fig, axs = plt.subplots(1, 3, figsize=(18, 5))

axs[0].bar(x - 0.2, pip_results['time_sec'], 0.4, label='pip')
axs[0].bar(x + 0.2, uv_results['time_sec'], 0.4, label='uv')
axs[0].set_ylabel('Install Time (s)')
axs[0].set_xticks(x)
axs[0].set_xticklabels(labels, rotation=45)
axs[0].legend()
axs[0].set_title('Installation Time')

axs[1].bar(x - 0.2, pip_results['max_mem_mb'], 0.4, label='pip')
axs[1].bar(x + 0.2, uv_results['max_mem_mb'], 0.4, label='uv')
axs[1].set_ylabel('Max Memory Usage (MB)')
axs[1].set_xticks(x)
axs[1].set_xticklabels(labels, rotation=45)
axs[1].legend()
axs[1].set_title('Memory Usage')

axs[2].bar(x - 0.2, pip_results['max_cpu_pct'], 0.4, label='pip')
axs[2].bar(x + 0.2, uv_results['max_cpu_pct'], 0.4, label='uv')
axs[2].set_ylabel('Max CPU (%)')
axs[2].set_xticks(x)
axs[2].set_xticklabels(labels, rotation=45)
axs[2].legend()
axs[2].set_title('CPU Usage')

plt.tight_layout()
plt.show()